In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from bs4 import BeautifulSoup
import csv
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import nltk
!pip install nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
path = "/content/drive/MyDrive/MWP(Words and Values in List).csv"
df = pd.read_csv(path)
df.head()

,Unnamed: 0,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df = df.iloc[:,1:3]
df

,Problem,Level
0,There are constants alpha and beta suc...,Level 5
1,The length of the segment between the points ...,Level 5
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5
3,"For some constants a and b, let [f(x) = ...",Level 5
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2
...,...,...
1739,What is the slope of the line passing through ...,Level 2
1740,Evaluate i^{11} + i^{16} + i^{21} + i^{26} + ...,Level 4
1741,If a is a constant such that 4x^2 - 12x + a...,Level 3
1742,"Given a star b = a^2 + 2ab + b^2 , what is ...",Level 3


In [6]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [7]:
df.head()

,Problem,Level
0,There are constants alpha and beta suc...,5
1,The length of the segment between the points ...,5
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5
3,"For some constants a and b, let [f(x) = ...",5
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2


# Function to count number of sentences in the problem. 

In [8]:
def count_sentences(data):
  number_of_sentences = sent_tokenize(data)
  return len(number_of_sentences)

#Function to count number of words in the problem.

In [9]:
def count_words(data):
  count = 0
  for i in data:
    for j in i:
      count = count+1
  return count

#Function to count number of words in the problem.

In [10]:
def count_chars(data):
  number_of_words = len(data.split())
  return number_of_words

#Function to count syllables


In [11]:
#Counts syllables in every word
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [12]:
def count_syllables(data):
  total_count = 0
  data = data.split()
  for word in data: 
    #print(word)
    temp = syllable_count(word)
    total_count = total_count+temp
  return total_count

In [13]:
def count_poly_syllables(data):
  total_count = 0
  data = data.split()
  for word in data: 
    #print(word)
    temp = syllable_count(word)
    if temp>=3:
      total_count = total_count+temp
  return total_count

# Complex Words

In [14]:
path = "/content/drive/MyDrive/Math Words.csv"
df_cw = pd.read_csv(path)
df_cw

,Unnamed: 0,Word,Value
0,0,x,2675
1,1,What,597
2,2,frac,573
3,3,f,544
4,4,b,518
...,...,...,...
2253,2253,assignment,1
2254,2254,rightarrow,1
2255,2255,Janice,1
2256,2256,farther,1


In [15]:
df_cw = df_cw.iloc[:,1]
df_cw.head()

0       x
1    What
2    frac
3       f
4       b
Name: Word, dtype: object

In [16]:
#Say we choose words of length >6 as complex words. 
complex_words = []
for i in df_cw:
  if len(i)>6:
    complex_words.append(i)

In [17]:
len(complex_words)

880

In [18]:
def count_cw(data):
  total_count = 0
  data = data.split()
  for word in data: 
    if word in complex_words:
      total_count = total_count+1
  return total_count

#Gunning Fog Index

In [19]:
# 0.4[ (words/sentence) + 100(complex words/words) ]
GF = []
for i in df.iloc[:,0]:
  w = count_words(i)
  c = count_cw(i)
  s = count_sentences(i)
  gf_score = (w/s) + (100*c/w)
  GF.append(0.4*gf_score)

In [20]:
len(GF)

1744

#Flesch - Kincaid Readability Index

In [21]:
#Formula: 206.835 - 1.015(total words/total sentences) - 84.6(total syllables/total words)
FK = []
for i in df.iloc[:,0]:
  w = count_words(i)
  syl = count_syllables(i)
  s = count_sentences(i)
  fk_score = (1.015*w/s) + (84.6*syl/w)
  fk_score = 206.835 - fk_score 
  FK.append(fk_score)

In [22]:
len(FK)

1744

#SMOG (Simple Measure of Gobbledygook) Index

In [23]:
# Fromula: 3.1291 + 1.0430(sqrt(number of polysyllables * 30 /number of sentences))
SMOG = []
for i in df.iloc[:,0]:
  syl = count_poly_syllables(i)
  s = count_sentences(i)
  smog_score = np.sqrt(30*syl/s)
  smog_score = smog_score * 1.0430
  SMOG.append(3.1291+smog_score)

In [24]:
len(SMOG)

1744

#Automated Readability Index

In [25]:
# Fromula: 4.71(characters/word) + 0.5(words/sentence) - 21.43

In [26]:
AR = []
for i in df.iloc[:,0]:
  w = count_words(i)
  c = count_chars(i)
  s = count_sentences(i)
  ar_score = (4.71*c/w) + (0.5*w/s) - 21.43
  AR.append(ar_score)

In [27]:
len(AR)

1744

# Coleman-Liau Index

In [28]:
#Formula: 0.0588(Avg characters per 100 words) - 15.8 - 0.296(average number of sentences per 100 words)
CL = []
for i in df.iloc[:,0]:
  w = count_words(i)
  c = count_chars(i)
  s = count_sentences(i)
  term1 = 0.0588*c*100/w #c:w :: ?:100
  term3 = 0.296*s*100/w #s:w :: ?:100
  cl_score = term1-15.8-term3
  CL.append(cl_score)

In [29]:
len(CL)

1744

#Compute Correlation

In [30]:
df["GF"]=GF
df["FK"]=FK
df["SMOG"]=SMOG
df["AR"]=AR
df["CL"]=CL
df.head()

,Problem,Level,GF,FK,SMOG,AR,CL
0,There are constants alpha and beta suc...,5,29.473973,117.674247,3.129100,15.682945,-15.440274
1,The length of the segment between the points ...,5,30.021918,113.038630,3.129100,15.973288,-15.077808
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,33.013069,102.416564,11.979248,19.914332,-14.987426
3,"For some constants a and b, let [f(x) = ...",5,33.156463,102.879660,13.559100,20.364558,-14.962449
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,20.800000,127.321000,13.023867,4.983000,-14.628000


In [31]:
np.corrcoef(GF, df.iloc[:,1])

array([[1.        , 0.20128413],
       [0.20128413, 1.        ]])

In [32]:
np.corrcoef(FK, df.iloc[:,1]) 

array([[ 1.        , -0.19783011],
       [-0.19783011,  1.        ]])

In [33]:
np.corrcoef(SMOG, df.iloc[:,1]) 

array([[1.        , 0.21108224],
       [0.21108224, 1.        ]])

In [34]:
np.corrcoef(AR, df.iloc[:,1]) 

array([[1.        , 0.19905358],
       [0.19905358, 1.        ]])

In [35]:
np.corrcoef(CL, df.iloc[:,1]) 

array([[1.        , 0.12532731],
       [0.12532731, 1.        ]])

#Conclusion

Correlation between any of the readability scores and level of difficulty is very low. The correlation scores are 0.2,-0.19,0.21,0.19,0.12 for GF, FK, SMOG, AR, CL readability scores respectively. 

#Correlation With Number of Math Words

It could be the case that easy problems are very long. If the problem is very long, the number of math words would still be more, although the problem isnt hard. Similarly, hard problems could also be short. We compute the average sentence length to account for this difference. 

In [36]:
#Compute average word problem length. 
len_lis = []
for i in df.iloc[:,0]:
  t = len(i.split())
  len_lis.append(t)

avg_len = sum(len_lis)/len(len_lis)
print(avg_len)

26.635894495412845


In [37]:
path = "/content/drive/MyDrive/Math Words Labelled.csv"
mw = pd.read_csv(path)
mw

,Unnamed: 0,Word,Value,Math Word (Label)
0,NaN,x,2675,NaN
1,NaN,What,597,NaN
2,NaN,frac,573,2
3,NaN,f,544,NaN
4,NaN,b,518,NaN
...,...,...,...,...
2253,NaN,assignment,1,NaN
2254,NaN,rightarrow,1,NaN
2255,NaN,Janice,1,NaN
2256,NaN,farther,1,NaN


In [38]:
math_words_list = []

In [39]:
for ind in mw.index:
  if(mw["Math Word (Label)"][ind]=="1"):
    math_words_list.append(mw["Word"][ind])

In [40]:
math_words_list

['value',
 'real',
 'answer',
 'many',
 'two',
 'integers',
 'equation',
 'positive',
 'graph',
 'number',
 'numbers',
 'integer',
 'values',
 'points',
 'square',
 'sequence',
 'size',
 'fraction',
 'line',
 'ymin',
 'ymax',
 'one',
 'three',
 'first',
 'term',
 'find',
 'product',
 'Evaluate',
 'linewidth',
 'function',
 'terms',
 'expression',
 'solutions',
 'axisarrowsize',
 'axispen',
 'feet',
 'area',
 'times',
 'coordinates',
 'Solve',
 'total',
 'units',
 'hours',
 'Size',
 'defined',
 'laxis',
 'Simplify',
 'distance',
 'height',
 'geometric',
 'miles',
 'pair',
 'difference',
 'quadratic',
 'array',
 'equal',
 'log',
 'bool',
 'arithmetic',
 'greatest',
 'second',
 'roots',
 'four',
 'interval',
 'rate',
 'width',
 'Compute',
 'domain',
 'interest',
 'smallest',
 'largest',
 'circle',
 'least',
 'axes',
 'xaxis',
 'yaxis',
 'length',
 'notation',
 'lines',
 'segment',
 'ratio',
 'solution',
 'five',
 'triangle',
 'cartesian',
 'squares',
 'range',
 'cost',
 'polynomial',
 'St

In [41]:
count_list = []
for i in df.iloc[:,0]:
  temp = 0
  for j in i.split():
    if j in math_words_list:
      temp = temp+1
  #Normalise the count value. 
  temp = temp*avg_len/len(i.split())
  count_list.append(temp)

In [42]:
count_list

[1.4018891839690972,
 4.756409731323722,
 1.3153528145882887,
 1.0654357798165137,
 1.7757262996941896,
 4.097829922371207,
 7.398859582059123,
 2.0489149611856035,
 2.6635894495412846,
 4.237528669724771,
 2.219657874617737,
 4.994230217889909,
 3.805127785058978,
 1.1098289373088686,
 2.219657874617737,
 7.610255570117956,
 4.439315749235474,
 5.327178899082569,
 10.328203988017226,
 3.3294868119266057,
 7.7688025611620795,
 0.0,
 5.607556735876389,
 2.0489149611856035,
 1.902563892529489,
 3.5047229599227427,
 4.994230217889909,
 4.842889908256881,
 5.327178899082569,
 6.905602276588515,
 1.7757262996941896,
 2.8684809456598446,
 6.948494216194655,
 2.8538458387942334,
 3.929886073093699,
 8.878631498470948,
 3.632167431192661,
 6.392614678899083,
 4.842889908256881,
 1.4018891839690972,
 6.267269293038317,
 1.8583182206101987,
 2.5367518567059855,
 0.0,
 4.632329477463103,
 2.0489149611856035,
 2.577667209233501,
 2.1743587343194157,
 4.994230217889909,
 2.8037783679381945,
 5.7076

In [43]:
np.corrcoef(count_list, df.iloc[:,1]) 

array([[1.        , 0.06658559],
       [0.06658559, 1.        ]])

In [46]:
#Without Normalisation
count_list = []
for i in df.iloc[:,0]:
  temp = 0
  for j in i.split():
    if j in math_words_list:
      temp = temp+1
  count_list.append(temp)
np.corrcoef(count_list, df.iloc[:,1]) 

array([[1.        , 0.29114796],
       [0.29114796, 1.        ]])

#Parts Of Speech Tags

In [ ]:
"""
CC coordinating conjunction 
CD cardinal digit 
DT determiner 
EX existential there (like: “there is” … think of it like “there exists”) 
FW foreign word 
IN preposition/subordinating conjunction 
JJ adjective – ‘big’ 
JJR adjective, comparative – ‘bigger’ 
JJS adjective, superlative – ‘biggest’ 
LS list marker 1) 
MD modal – could, will 
NN noun, singular ‘- desk’ 
NNS noun plural – ‘desks’ 
NNP proper noun, singular – ‘Harrison’ 
NNPS proper noun, plural – ‘Americans’ 
PDT predeterminer – ‘all the kids’ 
POS possessive ending parent’s 
PRP personal pronoun –  I, he, she 
PRP$ possessive pronoun – my, his, hers 
RB adverb – very, silently, 
RBR adverb, comparative – better 
RBS adverb, superlative – best 
RP particle – give up 
TO – to go ‘to’ the store. 
UH interjection – errrrrrrrm 
VB verb, base form – take 
VBD verb, past tense – took 
VBG verb, gerund/present participle – taking 
VBN verb, past participle – taken 
VBP verb, sing. present, non-3d – take 
VBZ verb, 3rd person sing. present – takes 
WDT wh-determiner – which 
WP wh-pronoun – who, what 
WP$ possessive wh-pronoun, eg- whose 
WRB wh-abverb, eg- where, when
"""

'\nCC coordinating conjunction \nCD cardinal digit \nDT determiner \nEX existential there (like: “there is” … think of it like “there exists”) \nFW foreign word \nIN preposition/subordinating conjunction \nJJ adjective – ‘big’ \nJJR adjective, comparative – ‘bigger’ \nJJS adjective, superlative – ‘biggest’ \nLS list marker 1) \nMD modal – could, will \nNN noun, singular ‘- desk’ \nNNS noun plural – ‘desks’ \nNNP proper noun, singular – ‘Harrison’ \nNNPS proper noun, plural – ‘Americans’ \nPDT predeterminer – ‘all the kids’ \nPOS possessive ending parent’s \nPRP personal pronoun –  I, he, she \nPRP$ possessive pronoun – my, his, hers \nRB adverb – very, silently, \nRBR adverb, comparative – better \nRBS adverb, superlative – best \nRP particle – give up \nTO – to go ‘to’ the store. \nUH interjection – errrrrrrrm \nVB verb, base form – take \nVBD verb, past tense – took \nVBG verb, gerund/present participle – taking \nVBN verb, past participle – taken \nVBP verb, sing. present, non-3d – 

In [47]:
def removePunct(data):
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for ele in data:
    if ele in punc:
        data = data.replace(ele, "")
  return data

In [48]:
def computePOS(txt):
  tokenized = sent_tokenize(txt)
  l = [] #Pos list for the word problem. 
  for i in tokenized:
    #i = each sentence in the word problem.
    i = removePunct(i) 
    wordsList = nltk.word_tokenize(i) #List of words in sentence i.
    wordsList = [w for w in wordsList if not w in stop_words] #Remove stop words. 
    tagged = nltk.pos_tag(wordsList)
    c = 0
    for i in tagged:
      l.append(tagged[c][1])
      c = c+1
  return l

In [49]:
path = "/content/drive/MyDrive/MWP_Sorted.csv" #Sorted the word problems in order of difficulty. 
df = pd.read_csv(path)
df = df.iloc[:,1:]
df.head()

,Problem,Level
0,Evaluate: 64^2-36^2,1
1,"A particular novel contains 40, !000 words....",1
2,Addison's age is three times Brenda's age. Jan...,1
3,Solve for x : dfrac{1}{2} + dfrac{1}{x} ...,1
4,Evaluate (x-a+3) if the value of x is (a+...,1


In [50]:
pos_list = []
for i in df.iloc[:,0]:
  l = computePOS(i)
  pos_list.append(l)

In [51]:
df["pos_tag"]=pos_list
df.head()

,Problem,Level,pos_tag
0,Evaluate: 64^2-36^2,1,"[NNP, CD]"
1,"A particular novel contains 40, !000 words....",1,"[DT, JJ, NN, VBZ, CD, CD, NNS, IN, NN, VBD, JJ..."
2,Addison's age is three times Brenda's age. Jan...,1,"[NNS, NN, CD, NNS, NNP, NN, NNP, CD, NNS, JJR,..."
3,Solve for x : dfrac{1}{2} + dfrac{1}{x} ...,1,"[NNP, NNP, NN, NNP, NN, NNP, NN]"
4,Evaluate (x-a+3) if the value of x is (a+...,1,"[NNP, NNP, NN, NNP, NN]"


In [52]:
#Idea: Creat trigrams of the pos tags. Find the count occurance of each in the each level of problems. 
#Choose the top 5 frequently occuring ones. 

In [53]:
df1 = df.iloc[:178,:] #Level 1 problems. 
df2 = df.iloc[178:355,:] #Level 2 problems. 
df3 = df.iloc[518:695,:] #Level 3 problems. 
df4 = df.iloc[910:1087,:] #Level 4 problems.
df5 = df.iloc[1308:1485,:] #Level 5 problems. 
#To compare the counts, we need all dfs to be of the same size. So, we take the minimum size, i.e., 177
#518, 392, 834

In [54]:
def create_trigram_list(data):
  l = []
  for i in range(0, len(data)-2):
    l.append(data[i]+" "+data[i+1]+" "+data[i+2])
  return l

In [55]:
def creat_count_dict(data): 
  #Data = df of one level of problems. 
  data = data.iloc[:,-1] #data = list of pos tags. 
  #For each problem, it sees trigram pos and increments the count in the dictionary. 
  postings = {}
  for i in data:
    if len(i)>=3:
      l = create_trigram_list(i)
      for j in l:
        if j not in postings: 
          postings[j] = 1
        else:
          postings[j] = postings[j] + 1

  return postings

In [56]:
l1 = creat_count_dict(df1) 
l2 = creat_count_dict(df2)
l3 = creat_count_dict(df3)
l4 = creat_count_dict(df4)
l5 = creat_count_dict(df5)
#Each is a dictionary. Key = trigrams of POS, value = count of POS in level i of problems. 

In [57]:
#Sort the dictionaries based on values,i.e., frequency of POS tags. 
sl1 = sorted(l1.items(), key =lambda kv:(kv[1], kv[0]))
sl1[len(sl1)-5:len(sl1)]

[('JJ NN NN', 26),
 ('CD CD CD', 32),
 ('CD NN NN', 32),
 ('NN CD NN', 32),
 ('NN NN NN', 39)]

In [58]:
sl2 = sorted(l2.items(), key =lambda kv:(kv[1], kv[0]))
sl2[len(sl2)-5:len(sl2)]

[('NN CD NN', 23),
 ('JJ NN NN', 25),
 ('NNP NN NN', 28),
 ('CD NN CD', 29),
 ('NN NN NN', 106)]

In [59]:
sl3 = sorted(l3.items(), key =lambda kv:(kv[1], kv[0]))
sl3[len(sl3)-5:len(sl3)]

[('NN NN NNP', 32),
 ('JJ NN NN', 34),
 ('NN NNP NN', 40),
 ('NNP NN NN', 49),
 ('NN NN NN', 92)]

In [60]:
sl4 = sorted(l4.items(), key =lambda kv:(kv[1], kv[0]))
sl4[len(sl4)-5:len(sl4)]

[('JJ NNS VBP', 34),
 ('JJ NN NN', 38),
 ('NNP NN NN', 43),
 ('NN NNP NN', 45),
 ('NN NN NN', 114)]

In [61]:
sl5 = sorted(l5.items(), key =lambda kv:(kv[1], kv[0]))
sl5[len(sl5)-5:len(sl5)]

[('NN JJ NN', 84),
 ('NN NNP NN', 88),
 ('JJ JJ NN', 94),
 ('JJ NN NN', 106),
 ('NN NN NN', 164)]

JJ NN NN, JJ JJ NN -> Found mostly in level 5 problems. 
Perhaps adjectives for the nouns makes the problems harder.

CD followed by NN (nouns along with simple cardinal digits) makes the problem easy. 